In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import spacy
nlp = spacy.load("ru_core_news_sm")

In [3]:
"""
 Read sales files with shop information. 
"""
def read_input_sales():
    
    df_sales = pd.read_csv("input/competitive-data-science-predict-future-sales/sales_train.csv")
    df_shop = pd.read_csv("input/competitive-data-science-predict-future-sales/shops.csv")
    
    # Create a word2vec for the shop name. It will be our categorical embeding for shop_id
    df_shop[['shop_vect_'+str(i) for i in range(0,96)]] = df_shop['shop_name'].apply(lambda x: nlp(x).vector).to_list()

    df_tmp = df_sales.join(df_shop.set_index("shop_id"),  on="shop_id")

    df_tmp['datetime'] = pd.to_datetime(df_tmp['date'], format='%d.%m.%Y')
    df_tmp['month'] = df_tmp['datetime'].dt.month
    df_tmp = df_tmp.drop(["date", "datetime"], axis=1)
    
    return df_tmp

read_input_sales()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,shop_vect_0,shop_vect_1,shop_vect_2,shop_vect_3,...,shop_vect_87,shop_vect_88,shop_vect_89,shop_vect_90,shop_vect_91,shop_vect_92,shop_vect_93,shop_vect_94,shop_vect_95,month
0,0,59,22154,999.00,1.0,"Ярославль ТЦ ""Альтаир""",0.407203,-0.583271,0.360839,0.672903,...,-0.252361,-0.301564,-0.85836,-0.831715,0.178552,-0.469958,-1.039193,-0.169419,0.870395,1
1,0,25,2552,899.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,1
2,0,25,2552,899.00,-1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,1
3,0,25,2554,1709.05,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,1
4,0,25,2555,1099.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,10
2935845,33,25,7460,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,10
2935846,33,25,7459,349.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,10
2935847,33,25,7440,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.076941,-0.560358,-0.55542,-1.124109,-0.240765,0.070050,-1.097628,0.115589,0.963545,10


In [6]:
"""
 Read all needed files and create this stage with raw entries before start dataset preparation
"""

def create_final_raw_dataset(skip=True):
    
    df = read_input_sales()
        
    df_item = pd.read_csv("input/competitive-data-science-predict-future-sales/items.csv")
    if not skip:
        df_item[['item_vect_'+str(i) for i in range(0,96)]] = df_item['item_name'].apply(lambda x: nlp(x).vector).to_list()
    df = df.join(df_item.set_index('item_id'), on="item_id")
    
    df_cat = pd.read_csv("input/competitive-data-science-predict-future-sales/item_categories.csv")
    if not skip:
        df_cat[['cat_vect_'+str(i) for i in range(0,96)]] = df_cat['item_category_name'].apply(lambda x: nlp(x).vector).to_list()
    df = df.join(df_cat.set_index("item_category_id"), on="item_category_id")
    
    return df

df_raw = create_final_raw_dataset(False)

# Filter invalid results in dataset
df_raw = df_raw[(df_raw['item_cnt_day'] > 0) & (df_raw['item_price'] > 0)]
df_raw

,date_block_num,shop_id,item_id,item_price,item_cnt_day,shop_name,shop_vect_0,shop_vect_1,shop_vect_2,shop_vect_3,...,cat_vect_86,cat_vect_87,cat_vect_88,cat_vect_89,cat_vect_90,cat_vect_91,cat_vect_92,cat_vect_93,cat_vect_94,cat_vect_95
0,0,59,22154,999.00,1.0,"Ярославль ТЦ ""Альтаир""",0.407203,-0.583271,0.360839,0.672903,...,0.435943,-0.407535,-0.653895,-0.972766,-0.909339,-0.191560,1.046879,-1.754487,-1.006972,-1.197602
1,0,25,2552,899.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.135546,-0.306363,0.052943,-0.630729,-0.988234,-0.794814,-0.354275,-0.540091,-0.905823,-0.583411
3,0,25,2554,1709.05,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.135546,-0.306363,0.052943,-0.630729,-0.988234,-0.794814,-0.354275,-0.540091,-0.905823,-0.583411
4,0,25,2555,1099.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-1.146514,-0.099345,-0.824269,-0.893807,-1.040153,-0.085812,1.099656,-1.356800,-0.250673,-1.029469
5,0,25,2564,349.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-2.488810,-0.229466,-0.598500,-0.011369,-1.473432,-0.322129,1.177074,-1.286037,0.650280,-0.795670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2935844,33,25,7409,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-1.308457,-0.219528,-0.800434,-1.119277,-0.883192,0.125871,1.123981,-1.359579,-0.002899,-1.213416
2935845,33,25,7460,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-1.308457,-0.219528,-0.800434,-1.119277,-0.883192,0.125871,1.123981,-1.359579,-0.002899,-1.213416
2935846,33,25,7459,349.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-1.308457,-0.219528,-0.800434,-1.119277,-0.883192,0.125871,1.123981,-1.359579,-0.002899,-1.213416
2935847,33,25,7440,299.00,1.0,"Москва ТРК ""Атриум""",0.639455,-0.949669,0.442484,0.786745,...,-0.827626,-0.817995,-0.575831,-0.971552,-0.831044,-0.416197,0.402561,-1.099714,-1.163974,-1.199133


# Feature Engineering

The follow lines we will insert more information to help in the prediction

In [7]:
def aggregate_final_dataset(df):

    agg_exp = {
        "item_cnt_day": "sum",
        "item_price": ["min","max", "mean", "median", "std"],
    }
    
    for i in range(0,10):
        agg_exp["cat_vect_"+str(i)] = "first"
        agg_exp["item_vect_"+str(i)] = "first"
        agg_exp["shop_vect_"+str(i)] = "first"
        
    df = df.groupby(["date_block_num", "month", "shop_id", "item_category_id","item_id"]).agg(agg_exp)
    
    df = df.reset_index()
    df.columns = [' '.join(col).strip() for col in df.columns.values]

    return df
    
df_final = aggregate_final_dataset(df_raw)

In [8]:
def fill_blank_dates(df):
    
    df_complete = pd.read_csv('input/competitive-data-science-predict-future-sales/sample_submission.csv', index_col='ID')
    df_complete['tmp'] = 1

    df_block = df.groupby(["date_block_num"]).first("month")
    df_block = df_block.reset_index()
#     df_block = df_block[['date_block_num', 'month']]
    df_block['tmp'] = 1
    
#     df_complete = pd.merge(df_complete, df_block, on=["tmp"], how="outer")
#     df_complete = df_complete.drop("tmp", axis=1)
    
    return df_block

fill_blank_dates(df_final)

,date_block_num,month,shop_id,item_category_id,item_id,item_cnt_day sum,item_price min,item_price max,item_price mean,item_price median,...,cat_vect_7 first,item_vect_7 first,shop_vect_7 first,cat_vect_8 first,item_vect_8 first,shop_vect_8 first,cat_vect_9 first,item_vect_9 first,shop_vect_9 first,tmp
0,0,1,0,2,5572,10.0,1322.00,1322.00,1322.00,1322.00,...,1.471781,-0.113999,0.562109,0.125695,0.076038,0.006519,1.277112,-0.442892,-0.383230,1
1,1,2,0,0,16255,1.0,93.00,93.00,93.00,93.00,...,0.360248,0.276076,0.562109,-0.824266,-0.038456,0.006519,-1.247780,0.112881,-0.383230,1
2,2,3,2,2,5572,1.0,1490.00,1490.00,1490.00,1490.00,...,1.471781,-0.113999,0.635260,0.125695,0.076038,1.101562,1.277112,-0.442892,-0.878841,1
3,3,4,2,2,5572,2.0,1490.00,1490.00,1490.00,1490.00,...,1.471781,-0.113999,0.635260,0.125695,0.076038,1.101562,1.277112,-0.442892,-0.878841,1
4,4,5,2,2,5571,1.0,499.00,499.00,499.00,499.00,...,1.471781,-0.066912,0.635260,0.125695,-0.000971,1.101562,1.277112,-0.749107,-0.878841,1
5,5,6,2,2,5572,2.0,1490.00,1490.00,1490.00,1490.00,...,1.471781,-0.113999,0.635260,0.125695,0.076038,1.101562,1.277112,-0.442892,-0.878841,1
6,6,7,2,2,5572,1.0,1590.00,1590.00,1590.00,1590.00,...,1.471781,-0.113999,0.635260,0.125695,0.076038,1.101562,1.277112,-0.442892,-0.878841,1
7,7,8,2,2,5574,2.0,499.00,499.00,499.00,499.00,...,1.471781,0.484841,0.635260,0.125695,0.486657,1.101562,1.277112,-0.462371,-0.878841,1
8,8,9,2,2,5575,1.0,999.00,999.00,999.00,999.00,...,1.471781,-0.168947,0.635260,0.125695,-0.021860,1.101562,1.277112,-0.738387,-0.878841,1
9,9,10,2,2,5572,2.0,1490.00,1590.00,1540.00,1540.00,...,1.471781,-0.113999,0.635260,0.125695,0.076038,1.101562,1.277112,-0.442892,-0.878841,1


In [9]:
def create_target(df):

    df_tmp = df[['date_block_num',"shop_id", "item_id", "item_cnt_day sum"]]
    df_tmp["date_block_num"] = df_tmp['date_block_num'] -1
    df_tmp = df_tmp.rename(columns = {'item_cnt_day sum': 'y'})
    df_tmp = df.join(df_tmp.set_index(['date_block_num',"shop_id", "item_id"]), on=['date_block_num',"shop_id", "item_id"], how="left")
    df_tmp = df_tmp.fillna(0)
    df_tmp['y'] = df_tmp['y'].clip(0,20)
    df_tmp['cliped_item_cnt'] = df_tmp['item_cnt_day sum'].clip(0,20)
    
    return df_tmp
    

df_final = create_target(df_final)

/tmp/ipykernel_29881/3884752388.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp["date_block_num"] = df_tmp['date_block_num'] -1


In [10]:
def mean_item_values(df, df_raw):
    columns = ['item_id', 'item_price', 'item_cnt_day']
    df_avg_item = df_raw[columns].groupby("item_id").mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_item_price", "item_cnt_day": "mean_item_cnt"})

    df = df.join(df_avg_item.set_index("item_id")[["mean_item_price", "mean_item_cnt"]], on="item_id", how="left")
    
    df_avg_item = df_raw[ ['date_block_num'] + columns].groupby(["date_block_num","item_id"]).mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_item_price_month", "item_cnt_day": "mean_item_cnt_month"})

    df = df.join(df_avg_item.set_index(["date_block_num","item_id"])[["mean_item_price_month", "mean_item_cnt_month"]], on=["date_block_num","item_id"], how="left")
    
    return df

df_final = mean_item_values(df_final, df_raw)

In [11]:
def mean_shop_values(df, df_raw):
    
    columns = ['shop_id', 'item_price', 'item_cnt_day']
    df_avg_item = df_raw[columns].groupby("shop_id").mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_shop_price", "item_cnt_day": "mean_shop_cnt"})

    df_ = df.join(df_avg_item.set_index("shop_id")[["mean_shop_price", "mean_shop_cnt"]], on="shop_id", how="left")
    
    df_avg_item = df_raw[ ['date_block_num'] + columns].groupby(["date_block_num","shop_id"]).mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_shop_price_month", "item_cnt_day": "mean_shop_cnt_month"})

    df = df.join(df_avg_item.set_index(["date_block_num","shop_id"])[["mean_shop_price_month", "mean_shop_cnt_month"]], on=["date_block_num","shop_id"], how="left")
    
    return df

df_final = mean_shop_values(df_final, df_raw)

In [12]:
def mean_category_values(df, df_raw):
    
    columns = ['item_category_id', 'item_price', 'item_cnt_day']
    df_avg_item = df_raw[columns].groupby(["item_category_id"]).mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_category_price", "item_cnt_day": "mean_category_cnt"})

    df = df.join(df_avg_item.set_index(["item_category_id"])[["mean_category_price", "mean_category_cnt"]], on=["item_category_id"], how="left")
    
    df_avg_item = df_raw[ ['date_block_num'] + columns].groupby(["date_block_num","item_category_id"]).mean()
    df_avg_item = df_avg_item.reset_index()
    df_avg_item = df_avg_item.rename(columns={"item_price": "mean_category_price_month", "item_cnt_day": "mean_category_cnt_month"})

    df = df.join(df_avg_item.set_index(["date_block_num","item_category_id"])[["mean_category_price_month", "mean_category_cnt_month"]], on=["date_block_num","item_category_id"], how="left")
    
    return df

df_final = mean_category_values(df_final, df_raw)

In [13]:
def cycle_features(df):
    
    df['month_sin'] = np.sin((df['month']-1)*(2*np.pi/12))
    df['month_cos'] = np.cos((df['month']-1)*(2*np.pi/12))
    
    return df.drop("month", axis=1)

df_final = cycle_features(df_final)

In [14]:
def lag_features(df, lag_size=3):
    
    columns = []
    for i in range(1, lag_size+1):
        new_column = 'lag_item_cnt_'+str(i)
        columns.append(new_column)
        index = ['date_block_num',"shop_id", "item_id"]
        
        df_tmp = df[ index + ["item_cnt_day sum"]]
        df_tmp["date_block_num"] = df_tmp['date_block_num'] +i
        df_tmp = df_tmp.rename(columns = {'item_cnt_day sum': new_column})
        df_tmp = df_tmp[index + [new_column]]
        df = df.join(df_tmp.set_index(index), on=index, how="left")
        df[f'l{i}-rate'] = df['item_cnt_day sum']/df[new_column]
            
    
    df['last_cnt_mean'] = df[['item_cnt_day sum'] + columns].mean(axis=1)
    
    return df

lag_size = 3
df_final = lag_features(df_final, lag_size)
df_final = df_final[df_final['date_block_num'] >= lag_size]
df_final = df_final.fillna(0)

/tmp/ipykernel_29881/2574289463.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_tmp["date_block_num"] = df_tmp['date_block_num'] +i


In [15]:
df_final = df_final[~((df_final[["l1-rate","l2-rate","l3-rate"]].mean(axis=1) > 10) & ((df_final[["l1-rate","l2-rate","l3-rate"]] == 0).sum(axis=1) == 0))]

# Experiment models

In [16]:
i_min = 27   # Start validation date, than we will iterate to have an average of values beteween 27 and 31
i_max = 32   # Last date_block_num with y
i_final = 33 # date_block_num that does not have an answer (y)
df_final = df_final.fillna(0)


target_column = 'y'
index_column = ['item_id', 'shop_id', 'item_category_id']
# index_column += ['item_price min']
# target_column = 'item_cnt_day sum'

# df_final['y'] = df_final['item_cnt_day sum']
df_final = df_final.fillna(0)

X_test = df_final[df_final['date_block_num'] == i_max].drop([target_column] + index_column, axis=1)
y_test = df_final[df_final['date_block_num'] == i_max][target_column]

X_final = df_final[df_final['date_block_num'] == 33].drop([target_column] + index_column, axis=1)

from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_squared_error
from sklearn import preprocessing

import numpy as np

   
models = [
#     ("Baseline", DummyRegressor()),
    ("LR", LinearRegression()),
#     ("RF", RandomForestRegressor(n_jobs=-1)),
#     ("XGBR_v1", XGBRegressor(
#                 max_depth=8,
#                 n_estimators=1000,
#                 min_child_weight=300, 
#                 colsample_bytree=0.8, 
#                 subsample=0.8, 
#                 eta=0.3,    
#                 seed=42)
#             ),
]

scaler = preprocessing.StandardScaler()
scaler_feature = [] 
scaler_feature.append("item_cnt_day sum")
scaler_feature.append("item_price min")
scaler_feature.append("item_price max")
scaler_feature.append("item_price mean")
scaler_feature.append("item_price median")
scaler_feature.append("item_price std")
scaler_feature.append("lag_item_cnt_1")
scaler_feature.append("l1-rate")
scaler_feature.append("lag_item_cnt_2")
scaler_feature.append("l2-rate")
scaler_feature.append("lag_item_cnt_3")
scaler_feature.append("l3-rate")
scaler_feature.append("last_cnt_mean")
scaler_feature.append("mean_item_price")
scaler_feature.append("mean_item_cnt")
scaler_feature.append('mean_item_price_month')
scaler_feature.append('mean_item_cnt_month')
scaler_feature.append('mean_shop_price_month')
scaler_feature.append('mean_shop_cnt_month')
scaler_feature.append('mean_category_price')
scaler_feature.append('mean_category_cnt')
scaler_feature.append('mean_category_price_month')
scaler_feature.append('mean_category_cnt_month')
scaler.fit(df_final[scaler_feature])

for model_name, model in models:
    print("========================")
    print(" Model:",model_name)
    print("========================")
    results = []
    for i in range(i_min, i_max):
    
        X_train = df_final[df_final['date_block_num'] < i].drop([target_column] + index_column, axis=1)
        y_train = df_final[df_final['date_block_num'] < i][target_column]

        X_valid = df_final[df_final['date_block_num'] == i].drop([target_column] + index_column, axis=1)
        y_valid = df_final[df_final['date_block_num'] == i][target_column]
        
        
        X_train[scaler_feature] = scaler.transform(X_train[scaler_feature])
        model.fit(X_train, y_train)
        
        X_valid[scaler_feature] = scaler.transform(X_valid[scaler_feature])
        y_pred = model.predict(X_valid)
        
        score = mean_squared_error(y_pred.clip(0,20), y_valid, squared=False)
        results.append(score)
        print("Train Iteration "+str(i-i_min)+":", score)   
    
    X_test[scaler_feature] = scaler.transform(X_test[scaler_feature])
    y_pred = model.predict(X_test)
    
    print("")
    print(model_name)
    print("Train Mean:", np.mean(results))
    print("Test result:", mean_squared_error(y_test, y_pred.clip(0,20), squared=False))
    print("")

 Model: LR
Train Iteration 0: 1.553027997025202
Train Iteration 1: 1.5758546359483039
Train Iteration 2: 1.5066483905018742
Train Iteration 3: 1.5837793429909328
Train Iteration 4: 1.4963049659054997

LR
Train Mean: 1.5431230664743627
Test result: 1.617718526684302



In [17]:
df_importance = pd.DataFrame([models[0][1].coef_])
# df_importance = pd.DataFrame([models[1][1].feature_importances_])
df_importance.columns = X_train.columns
df_importance.T.sort_values(0)

,0
item_price min,-2.072258
mean_item_price_month,-0.298689
cat_vect_3 first,-0.275162
shop_vect_1 first,-0.263534
mean_item_price,-0.166010
shop_vect_2 first,-0.165173
shop_vect_9 first,-0.159728
shop_vect_8 first,-0.138640
item_price std,-0.132882
cat_vect_0 first,-0.120323


In [18]:
import math

X_test['y'] = y_test
X_test['pred'] = y_pred.astype(int)
X_test['diff'] = abs(y_test - y_pred.astype(int))
# X_test.sort_values("diff")
X_test[(X_test['y'] == 0) & (X_test['diff'] > 10)].sort_values("diff")

,date_block_num,item_cnt_day sum,item_price min,item_price max,item_price mean,item_price median,item_price std,cat_vect_0 first,item_vect_0 first,shop_vect_0 first,...,lag_item_cnt_1,l1-rate,lag_item_cnt_2,l2-rate,lag_item_cnt_3,l3-rate,last_cnt_mean,y,pred,diff
1547866,32,1.779818,0.189685,0.298557,0.196567,0.182809,0.764842,0.401668,0.036832,0.136572,...,0.542840,2.342043,-0.050767,15.178007,-0.166660,-0.486391,0.661915,0.0,11,11.0
1569533,32,3.229172,-0.351527,-0.352210,-0.352030,-0.351643,-0.111343,0.185608,1.789832,0.363891,...,1.392339,1.731208,-0.176207,-0.419133,-0.166660,-0.486391,2.235841,0.0,11,11.0
1566118,32,2.504495,-0.492077,-0.489051,-0.490872,-0.490437,-0.111343,0.582656,1.208212,-0.160966,...,1.756410,0.927051,0.200111,6.614871,0.216530,8.645108,1.055397,0.0,11,11.0
1565915,32,2.746054,2.987327,2.898529,2.946253,2.945504,-0.111343,1.322384,1.285212,-0.160966,...,-0.185301,-0.530180,-0.176207,-0.419133,-0.166660,-0.486391,2.720126,0.0,11,11.0
1565857,32,2.142157,-0.033541,0.421775,0.119258,-0.037315,4.815555,1.628550,0.765085,-0.160966,...,1.392339,1.029398,0.576429,2.639129,3.537507,0.335034,1.751556,0.0,11,11.0
1562687,32,11.804519,0.316879,0.422394,0.313490,0.308414,0.425209,0.401668,0.055179,0.614102,...,0.542840,16.365253,0.325550,22.517838,-0.166660,-0.486391,4.132623,0.0,11,11.0
1556094,32,8.543471,0.444074,0.546232,0.441295,0.434020,0.526175,0.401668,0.002366,1.269802,...,0.664197,10.041534,-0.176207,-0.419133,-0.038930,86.461359,2.962268,0.0,11,11.0
1574357,32,2.021377,0.316879,0.422394,0.320820,0.308414,0.719879,0.401668,0.055179,0.014046,...,0.057413,9.100217,-0.176207,-0.419133,-0.166660,-0.486391,0.964593,0.0,11,11.0
1552630,32,6.127881,0.444074,0.546232,0.441757,0.434020,0.545796,0.401668,0.002366,0.170972,...,0.057413,26.333558,-0.176207,-0.419133,-0.166660,-0.486391,3.022804,0.0,11,11.0
1553150,32,4.074629,0.316879,0.422394,0.317230,0.308414,0.591168,0.401668,0.055179,0.455226,...,0.421483,6.768647,-0.176207,-0.419133,0.088800,20.952781,1.428700,0.0,11,11.0


In [19]:
(df_final[["l1-rate","l2-rate","l3-rate"]] == 0).sum(axis=1) > 0

187032     False
187033      True
187034     False
187035      True
187036      True
           ...  
1608221     True
1608222    False
1608223    False
1608224    False
1608225     True
Length: 1420892, dtype: bool

In [20]:
df_final[(df_final[["l1-rate","l2-rate","l3-rate"]].mean(axis=1) > 10) & ((df_final[["l1-rate","l2-rate","l3-rate"]] == 0).sum(axis=1) == 0)][250:302]

,date_block_num,shop_id,item_category_id,item_id,item_cnt_day sum,item_price min,item_price max,item_price mean,item_price median,item_price std,...,mean_category_cnt_month,month_sin,month_cos,lag_item_cnt_1,l1-rate,lag_item_cnt_2,l2-rate,lag_item_cnt_3,l3-rate,last_cnt_mean


In [21]:
df_final[(df_final["item_id"] == 2969) & (df_final["shop_id"] == 31)]

,date_block_num,shop_id,item_category_id,item_id,item_cnt_day sum,item_price min,item_price max,item_price mean,item_price median,item_price std,...,mean_category_cnt_month,month_sin,month_cos,lag_item_cnt_1,l1-rate,lag_item_cnt_2,l2-rate,lag_item_cnt_3,l3-rate,last_cnt_mean
438028,7,31,30,2969,16.0,499.0,549.0,505.250000,499.0,17.677670,...,1.328858,-5.000000e-01,-8.660254e-01,0.0,0.000000,0.0,0.000000,0.0,0.000000,16.000000
492804,8,31,30,2969,6.0,549.0,549.0,549.000000,549.0,0.000000,...,1.508149,-8.660254e-01,-5.000000e-01,16.0,0.375000,0.0,0.000000,0.0,0.000000,11.000000
544557,9,31,30,2969,1.0,549.0,549.0,549.000000,549.0,0.000000,...,1.470134,-1.000000e+00,-1.836970e-16,6.0,0.166667,16.0,0.062500,0.0,0.000000,7.666667
595148,10,31,30,2969,2.0,439.2,549.0,494.100000,494.1,77.640325,...,1.392810,-8.660254e-01,5.000000e-01,1.0,2.000000,6.0,0.333333,16.0,0.125000,6.250000
654404,11,31,30,2969,7.0,208.0,208.6,208.120000,208.0,0.268328,...,1.284046,-5.000000e-01,8.660254e-01,2.0,3.500000,1.0,7.000000,6.0,1.166667,4.000000
713864,12,31,30,2969,6.0,173.6,208.0,202.266667,208.0,14.043741,...,1.208094,0.000000e+00,1.000000e+00,7.0,0.857143,2.0,3.000000,1.0,6.000000,4.000000
764634,13,31,30,2969,4.0,173.0,173.6,173.300000,173.3,0.346410,...,1.242373,5.000000e-01,8.660254e-01,6.0,0.666667,7.0,0.571429,2.0,2.000000,4.750000
811996,14,31,30,2969,5.0,173.0,173.6,173.300000,173.3,0.346410,...,1.295059,8.660254e-01,5.000000e-01,4.0,1.250000,6.0,0.833333,7.0,0.714286,5.500000
859235,15,31,30,2969,3.0,99.0,124.0,107.333333,99.0,14.433757,...,1.203261,1.000000e+00,6.123234e-17,5.0,0.600000,4.0,0.750000,6.0,0.500000,4.500000
904251,16,31,30,2969,2.0,74.0,99.0,86.500000,86.5,17.677670,...,1.199278,8.660254e-01,-5.000000e-01,3.0,0.666667,5.0,0.400000,4.0,0.500000,3.500000


In [22]:
df_final[(df_final["y"] > 100) & (df_final["date_block_num"] == 7 )][["date_block_num","shop_id", "item_id","y"]]

,date_block_num,shop_id,item_id,y


# Model Submission

In [23]:
sample_submission = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv', index_col='ID')
sample_submission
# prediction = model.predict(X_final)
# sample_submission['item_cnt_month'] = np.rint(prediction)
# sample_submission['item_cnt_month']

FileNotFoundError: [Errno 2] No such file or directory: '../input/competitive-data-science-predict-future-sales/sample_submission.csv'

In [ ]:
sample_submission.to_csv('submission.csv')

In [ ]:
X_valid.columns